In [1]:
import json, pprint, requests

In [2]:
host = 'http://ec2-3-126-91-209.eu-central-1.compute.amazonaws.com:8998'
file = 's3://aws-logs-832495690894-eu-central-1/rumble/rumbledb-1.15.0-with-query-parameter.jar'

In [3]:
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [40]:
from IPython.core.magic import register_line_cell_magic
import time
from pprint import pprint

def spark_submit(host, command, file):
    data = {
        'file': file,
        'className': 'org.rumbledb.cli.Main',
        'args': ['--master', 'yarn-cluster', '--query', command, '--show-error-info', 'yes']
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(host + '/batches', data = json.dumps(data), headers=headers)
    return response.headers['Location']

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell
    
    start = time.time()
    location = spark_submit(host, data, file)
    r = requests.get(host + location, data=json.dumps({'kind': 'spark'}), headers={'Content-Type': 'application/json'})
    while True:
        r = requests.get(host + location + '/state', data=json.dumps({'kind': 'spark'}), headers={'Content-Type': 'application/json'})
        state = r.json()['state']
        if state == 'success' or state == 'dead':
            break
        time.sleep(0.1)
    end = time.time()
    r = requests.get(host + location + '/log', data=json.dumps({'kind': 'spark'}), params={'from': 0, 'size': 99999}, headers={'Content-Type': 'application/json'})
    response = r.json()
    print("Took: %s ms" % (end - start))
    pprint(response['log'])

In [45]:
%%rumble
for $i in 1 to 100
return $i * 2

Job sent
Took: 4.888845920562744 ms
['stdout: ',
 '2',
 '4',
 '6',
 '8',
 '10',
 '12',
 '14',
 '16',
 '18',
 '20',
 '22',
 '24',
 '26',
 '28',
 '30',
 '32',
 '34',
 '36',
 '38',
 '40',
 '42',
 '44',
 '46',
 '48',
 '50',
 '52',
 '54',
 '56',
 '58',
 '60',
 '62',
 '64',
 '66',
 '68',
 '70',
 '72',
 '74',
 '76',
 '78',
 '80',
 '82',
 '84',
 '86',
 '88',
 '90',
 '92',
 '94',
 '96',
 '98',
 '100',
 '102',
 '104',
 '106',
 '108',
 '110',
 '112',
 '114',
 '116',
 '118',
 '120',
 '122',
 '124',
 '126',
 '128',
 '130',
 '132',
 '134',
 '136',
 '138',
 '140',
 '142',
 '144',
 '146',
 '148',
 '150',
 '152',
 '154',
 '156',
 '158',
 '160',
 '162',
 '164',
 '166',
 '168',
 '170',
 '172',
 '174',
 '176',
 '178',
 '180',
 '182',
 '184',
 '186',
 '188',
 '190',
 '192',
 '194',
 '196',
 '198',
 '200',
 '21/10/14 14:00:29 INFO ShutdownHookManager: Shutdown hook called',
 '21/10/14 14:00:29 INFO ShutdownHookManager: Deleting directory '
 '/mnt/tmp/spark-b106db8f-f4a4-486d-a6b7-ad69cc2cf584',
 '\nstderr: 

In [46]:
%%rumble
{ "pi" : 3.14, "sq2" : 1.4 }

Job sent
Took: 4.884994029998779 ms
['stdout: ',
 '{ "pi" : 3.14, "sq2" : 1.4 }',
 '21/10/14 14:00:47 INFO ShutdownHookManager: Shutdown hook called',
 '21/10/14 14:00:47 INFO ShutdownHookManager: Deleting directory '
 '/mnt/tmp/spark-6abe2ad5-c423-4b45-a2c2-98334a8c439b',
 '\nstderr: ']


In [47]:
%%rumble
[ 2, 3, 5, 7, 11, 13 ]

Job sent
Took: 4.801717042922974 ms
['stdout: ',
 '[ 2, 3, 5, 7, 11, 13 ]',
 '21/10/14 14:01:13 INFO ShutdownHookManager: Shutdown hook called',
 '21/10/14 14:01:13 INFO ShutdownHookManager: Deleting directory '
 '/mnt/tmp/spark-9ae9c8f1-d982-4d9b-9730-4dbbbc4de63f',
 '\nstderr: ']


In [133]:
%%rumble
{
      "operations" : [
        { "binary" : [ "and", "or"] },
        { "unary" : ["not"] }
      ],
      "bits" : [
        0, 1
      ]
    }

Job sent
Took: 5.626327991485596 ms
('{ "operations" : [ { "binary" : [ "and", "or" ] }, { "unary" : [ "not" ] } '
 '], "bits" : [ 0, 1 ] }\n'
 '21/10/10 21:17:52 INFO ShutdownHookManager: Shutdown hook called')


In [134]:
%%rumble
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]

Job sent
Took: 5.612520933151245 ms
('[ { "Question" : "Ultimate" }, [ "Life", "the universe", "and everything" ] '
 ']\n'
 '21/10/10 21:17:59 INFO ShutdownHookManager: Shutdown hook called')


In [135]:
%%rumble
"Hello, World!"

Job sent
Took: 5.587418079376221 ms
('Hello, World!\n'
 '21/10/10 21:18:05 INFO ShutdownHookManager: Shutdown hook called')


In [11]:
%%rumble
let $x := 1
let $x := $x + 2
let $x := $x + 3
return $x

Job sent
Took: 4.845235109329224 ms
['stdout: ',
 '6',
 '21/10/14 13:33:22 INFO ShutdownHookManager: Shutdown hook called',
 '21/10/14 13:33:22 INFO ShutdownHookManager: Deleting directory '
 '/mnt/tmp/spark-6dac6250-abb6-4e23-a1e8-bd5f4a43c50d',
 '\nstderr: ']


In [44]:
%%rumble
let $friends := [ "Jim", "Mary", "Jennifer"]
return $friends[[1+1]]

Job sent
Took: 4.712280035018921 ms
['stdout: ',
 'Mary',
 '21/10/14 13:52:13 INFO ShutdownHookManager: Shutdown hook called',
 '21/10/14 13:52:13 INFO ShutdownHookManager: Deleting directory '
 '/mnt/tmp/spark-51a91706-7d3c-40eb-a296-dc1d9503be4b',
 '\nstderr: ']


In [43]:
%%rumble
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "products" : [ distinct-values($product.product) ]
}

Job sent
Took: 27.104483127593994 ms
['stdout: ',
 '21/10/14 13:47:50 INFO S3NativeFileSystem: Opening '
 "'s3://aws-logs-832495690894-eu-central-1/rumble/rumbledb-1.15.0-with-query-parameter.jar' "
 'for reading',
 '{ "store" : 1, "products" : [ "shirt", "toaster", "phone", "blender", "tv", '
 '"socks", "broiler" ] }',
 '{ "store" : 2, "products" : [ "shirt", "toaster", "phone", "blender", "tv", '
 '"socks", "broiler" ] }',
 '{ "store" : 3, "products" : [ "shirt", "toaster", "phone", "blender", "tv", '
 '"socks", "broiler" ] }',
 '{ "store" : 4, "products" : [ "shirt", "toaster", "phone", "blender", "tv", '
 '"socks", "broiler" ] }',
 '{ "store" : 5, "products" : [ "shirt", "toaster", "phone", "blender", "tv", '
 '"socks", "broiler" ] }',
 '{ "store" : 6, "products" : [ "toaster", "phone", "blender", "tv", "socks", '
 '"broiler", "shirt" ] }',
 '{ "store" : 7, "products" : [ "toaster", "phone", "blender", "tv", "socks", '
 '"broiler", "shirt" ] }',
 '{ "store" : 8, "products" : [ "toa